In [1]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-61987d96-15d2-7202-6c40-63e37fe357fc)


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [3]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  proj_dir = "/content/drive/MyDrive/ece884_project/"
else:
  proj_dir = "../"

Mounted at /content/drive


In [4]:
df = pd.read_csv(f"{proj_dir}data_clean/clean.csv")
column_names = df.columns
df = df.to_numpy()

In [5]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(df) # dont forget we have to transfer them back
df = scaler.transform(df)

In [6]:

generator = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(200, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(400, activation="selu"),

    keras.layers.BatchNormalization(),
    keras.layers.Dense(400, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(200, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(98, activation="sigmoid")
])

In [7]:

discriminator = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(200, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(400, activation="selu"),
    keras.layers.BatchNormalization(),

    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(200, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1, activation="sigmoid")
])

In [8]:
gan = keras.models.Sequential([generator, discriminator])

discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [9]:
from tqdm import tqdm

In [10]:
def train_gan(
    gan, dataset, n_epochs=100, iterations=1000, n_noise=20000
):

    """
    Inputs: 

    gan, this is a keras gan object made by combining two neural nets and
    restricting the trainability of one of them.

    dataset, this takes in regular tabular data. now this is training rowwise
    however i may change this to matrix wise like a picture.

    n_epochs, numper of times the gans go though training iterations

    iterations, number of times in gan iteraton loop, 
    it would be a good idea to reduct this after the warmup period

    n_noise, this is the size of fake data generated

    
    Output:

    generators_saved, this is an iterable list of keras objects that can be used
    
    discriminators_saved, same thing, these can be used to test

    for generator, discriminator in zip(gen, desc):
        noise = tf.random.normal(shape=dims)
        generated_data = generator(noise)
        judgement = discriminator(generated_data) # probs data is real
    """
    generator, discriminator = gan.layers
    generators_saved = []
    discriminators_saved = []
    
    for epoch in range(n_epochs):
        print(f"Epoch {epoch} of {n_epochs}")
        if epoch > n_epochs / 4:  # give a 25% training period

            generators_saved.append(generator)
            discriminators_saved.append(discriminator)
            iterations = np.ceil(iterations / 1.1).astype('int') + 10 # after the training period reduce iterations

        for _ in tqdm(range(iterations)):
            random_index = np.random.permutation(len(dataset))
            X_batch = dataset[random_index, :]
            # phase 1 - training the discriminator

            noise = tf.random.normal(shape=(n_noise, 98),
                                     mean=0,
                                     stddev=0.1) 

            generated_data = generator(noise)
            X_fake_and_real = tf.concat([generated_data, X_batch], axis=0)

            y1 = tf.constant([[0.0]] * noise.shape[0] + [[1.0]] * X_batch.shape[0])
            y1 = np.reshape(y1, (len(y1), 1))

            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)
            # phase 2 - training the generator
            y2 = tf.constant([[1.0]] * n_noise)
            noise = tf.random.normal(shape=generated_data.shape)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)
        
        noise = tf.random.normal(shape=dims)
        generated_data = generator(noise)
        judgement = discriminator(generated_data) # probs data is real
        print(np.mean(judgement))

    return generators_saved, discriminators_saved

In [ ]:
generators_saved, discriminators_saved = train_gan(
    gan, df, n_epochs=100, iterations=10000, n_noise=20000
)

Epoch 0 of 100


 51%|█████▏    | 5126/10000 [10:10<09:27,  8.58it/s]

In [ ]:
import os
import re
import pickle
models = os.listdir(f"{proj_dir}saved_models/list_of_models/gen")
model_number = [int(re.sub("generators", "", x)) for x in models]
last_model = max(model_number)

In [ ]:
with open(f"{proj_dir}saved_models/list_of_models/gen/generators{last_model+1}", "wb") as fp:
    pickle.dump(generators_saved, fp)

with open(f"{proj_dir}saved_models/list_of_models/disc/discriminators{last_model+1}", "wb") as fp:
    pickle.dump(discriminators_saved, fp)